# Zadanie 4 – Analiza sentymentu tekstów użytkownika
W tym zadaniu budujemy klasyfikator, który rozpoznaje czy wypowiedź użytkownika ma charakter pozytywny, negatywny czy neutralny.

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

## Dane wejściowe

In [ ]:
data = {
    'text': [
        'Ten produkt jest świetny, polecam każdemu!',
        'To była najgorsza obsługa jaką kiedykolwiek miałem',
        'Jestem zadowolony z zakupu',
        'Nie polecam, bardzo słaba jakość',
        'Obsługa była w porządku, ale bez rewelacji',
        'Fantastyczne doświadczenie, wszystko poszło gładko',
        'Nie mogę powiedzieć, żeby to było dobre',
        'Zdecydowanie warto było!',
        'Raczej przeciętnie, spodziewałem się więcej',
        'Super, jestem bardzo zadowolony'
    ],
    'sentiment': [
        'pozytywny', 'negatywny', 'pozytywny', 'negatywny', 'neutralny',
        'pozytywny', 'negatywny', 'pozytywny', 'neutralny', 'pozytywny'
    ]
}
df = pd.DataFrame(data)

## Tokenizacja i kodowanie etykiet

In [ ]:
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded = pad_sequences(sequences, maxlen=10, padding='post')

label2id = {label: idx for idx, label in enumerate(df['sentiment'].unique())}
id2label = {idx: label for label, idx in label2id.items()}
labels = df['sentiment'].map(label2id).values

## Budowa i trening modelu

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=16, input_length=10),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(label2id), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

## Predykcja nowych wypowiedzi

In [ ]:
test_sentences = [
    'To było niesamowite przeżycie',
    'Jestem zawiedziony jakością produktu',
    'Nic specjalnego, tak sobie',
    'Obsługa klienta była wyjątkowa',
    'Nie warto tych pieniędzy'
]
seq = tokenizer.texts_to_sequences(test_sentences)
pad = pad_sequences(seq, maxlen=10, padding='post')
preds = model.predict(pad)

for i, p in enumerate(preds):
    print(f'{test_sentences[i]} -> {id2label[np.argmax(p)]}')